In [1]:
# Add testing url body text to csv to train a model:


In [2]:
import os, sys, re, time, gc, csv
read_loc = 'dataset/Raw/Read/'
store_loc = 'dataset/Raw/'
save_loc = f'{store_loc}LIWC/'

read_file = 'Fake' # URL_True, True, Fake

Notebook_start = time.time()

is_URL = False
sufix = 'base'
folders = ['base_body','base_title']
if 'URL' in read_file:
    is_URL = True
    sufix = 'url'
    folders = ['url_body','url_title']
    

from LIWC_Methods import Custom_Methods 
CM = Custom_Methods()

import pandas as pd
import numpy as np


In [3]:
print(f'{read_loc}{read_file}.csv')
df_old = pd.read_csv(f'{read_loc}{read_file}.csv',error_bad_lines=False, quotechar='"', thousands=',' ,low_memory=False)

# add id columns if not there
if 'id' not in list(df_old.columns):
    df_old['id'] = [f'{read_file}_{idx}' for idx in df_old.index]
    col_dict = {'text':'body'}
    keep_col = ['id','title','body']
    df_old.rename(columns=col_dict,inplace=True)
    df_old = df_old[keep_col]
if is_URL:    
    idx = list(df_old[df_old.body != ' '].index)
    df_old = df_old.loc[idx]
else:
    idx = list(df_old[df_old.body != ' '].index)
    df_old = df_old.loc[idx]
    idx = list(df_old[df_old.title != ' '].index)
    df_old = df_old.loc[idx]
    
    
df_old.sort_values(by=['id'],inplace=True)
df_old.reset_index(drop=True,inplace=True)
df_old['news_url'] = 'NA'
df_old['title_url_min'] = ''
df_old['body_url_min'] = ''
df_old['title_url'] = ''
df_old['body_url'] = ''
df_old['title_stem'] = ''
df_old['body_stem'] = ''
df_old['link_cnt'] = 0
df_old['process_state'] = 'Pass'
df_old

dataset/Raw/Read/Fake.csv


,id,title,body,news_url,title_url_min,body_url_min,title_url,body_url,title_stem,body_stem,link_cnt,process_state
0,Fake_0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,NA,,,,,,,0,Pass
1,Fake_1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,NA,,,,,,,0,Pass
2,Fake_10,"Papa John’s Founder Retires, Figures Out Raci...","A centerpiece of Donald Trump s campaign, and ...",NA,,,,,,,0,Pass
3,Fake_100,Watch: Joe Biden Just Wrecked Trump In Six Wo...,Former Vice President Joe Biden was asked on M...,NA,,,,,,,0,Pass
4,Fake_1000,Stunning New Poll Reveals Global Opinion Of D...,Donald Trump is doing a horrendous job of rais...,NA,,,,,,,0,Pass
...,...,...,...,...,...,...,...,...,...,...,...,...
22850,Fake_9995,LOL! SHAKEDOWN ARTIST JESSE JACKSON TELLS CROW...,Here s a 1999 video of Jesse Jackson praising ...,NA,,,,,,,0,Pass
22851,Fake_9996,President and First Lady Arrive in Texas: Serv...,Check out what s happening in Texas! President...,NA,,,,,,,0,Pass
22852,Fake_9997,Heaven’s Gatekeeper? Jesse Jackson Proclaims T...,Jesse Jackson thinks he s Heaven s gatekeeper ...,NA,,,,,,,0,Pass
22853,Fake_9998,First Lady Melania Trump Defies Critics…Rocks ...,The media will lose it again because Melania T...,NA,,,,,,,0,Pass


In [4]:
df_old.shape

(22855, 12)

In [5]:
cnt = 10000
# cnt = len(df_old)
df_old = df_old.sample(n=cnt, random_state=42)
df_old.reset_index(drop=True,inplace=True)
df_old.shape

(10000, 12)

In [6]:
i = 0
start = time.time()
last_start = start
save_text = True
    
end_cnt = len(df_old)
# end_cnt = 100


for idx, row in df_old[i:end_cnt].iterrows():
    if idx != -1:
        # print(idx,end=', ')
        gc.collect()
        if idx%250==0:
            last_start = CM.print_progress(idx,row,start,last_start)
        # try:
        if is_URL:
            row = CM.scrap_page(row)
        else:
            row = CM.clean_text(row)
        if ("page not found" in row.title_url) or ('page unavailable' in row.title_url) or ("scrapper blocked by website" in row.title_url):
            row.process_state = 'Failed'
            save_text = False
            # print(f'failed to clean text for idx:{idx}, ID: {row["id"]}')

        df = row.copy().to_frame().T
        if pd.notnull(df.body_url[idx]):
            if ((len(df.body_url[idx]) > 25) and (save_text == True)): # for LIWC Processing
                # print('Text Saved')
                for part in ['title','body']:
                    col = ['id',f'{part}_url']
                    id_ = df.id[idx]
                    CM.save_LIWC_csv(save_loc,idx,df,part,sufix, id_)
            else:
                # print(f'Text Not Saved for idx:{idx}, ID: {row["id"]}')
                row.process_state = 'Failed'
                save_text = True
        else:
            # print(f'df.body_url is null for idx:{idx}, ID: {row["id"]}')
            row.process_state = 'Failed'
            save_text = True
             
            
#         except:
#             print(f'failed to clean text for idx:{idx}, ID: {row["id"]}')
            
            
        CM.save_file(idx,df,read_file) 

end = time.time()
print(f'Total time in minutes: {int((end-start)/60)}', end=': ')
# 1000 < row < 1250

Total time:0  , Cycle time: 0.0, RowID:0 Attempting URL number:Fake_15289, URL: NA
Total time:1  , Cycle time: 1.4, RowID:250 Attempting URL number:Fake_15964, URL: NA
Total time:3  , Cycle time: 1.6, RowID:500 Attempting URL number:Fake_14820, URL: NA
Total time:5  , Cycle time: 1.6, RowID:750 Attempting URL number:Fake_19296, URL: NA
Total time:6  , Cycle time: 1.5, RowID:1000 Attempting URL number:Fake_11329, URL: NA
Total time:8  , Cycle time: 1.6, RowID:1250 Attempting URL number:Fake_14505, URL: NA
Total time:9  , Cycle time: 1.7, RowID:1500 Attempting URL number:Fake_4401, URL: NA
Total time:11  , Cycle time: 1.6, RowID:1750 Attempting URL number:Fake_13154, URL: NA
Total time:13  , Cycle time: 1.6, RowID:2000 Attempting URL number:Fake_11902, URL: NA
Total time:14  , Cycle time: 1.5, RowID:2250 Attempting URL number:Fake_5081, URL: NA
Total time:16  , Cycle time: 1.6, RowID:2500 Attempting URL number:Fake_20863, URL: NA
Total time:17  , Cycle time: 1.6, RowID:2750 Attempting UR

In [7]:
print('Done')

Done


In [9]:
# loc = f'{store_loc}Fake.csv'
# print(loc)
# df = pd.read_csv(loc)

# df.process_state = ['Pass' if len(str_)> 10 else 'Failed' for str_ in df.body_url]


In [10]:
loc

'dataset/Raw/Fake.csv'

In [11]:
# save file
df.to_csv(loc,mode='w', header=True, index=False)
df = pd.read_csv(loc)
df.head()

,id,title,body,news_url,title_url_min,body_url_min,title_url,body_url,title_stem,body_stem,link_cnt,process_state
0,Fake_15289,HILLARY’S CHICKENS ARE COMIN’ HOME TO ROOST: N...,Hillary may have gotten away with lying to the...,NaN,hillary’s chickens are comin’ home to roost: n...,hillary may have gotten away with lying to the...,hillary s chickens are comin home to roost new...,hillary may have gotten away with lying to the...,hillari chicken comin home roost news report s...,hillari may gotten away lie public decad under...,0,Pass
1,Fake_17685,DIAMOND and SILK Have Message For GOP Candidat...,After Ed Gillespie lost the VA Governor s race...,NaN,diamond and silk have message for gop candidat...,after ed gillespie lost the va governor s race...,diamond and silk have message for gop candidat...,after ed gillespie lost the va governor s race...,diamond silk messag gop candid embrac trump ed...,ed gillespi lost va governor race tonight pres...,0,Pass
2,Fake_6771,Fox News Says Non-Whites Should Only Be Depic...,Republicans found their new election year wedg...,NaN,fox news says non-whites should only be depic...,republicans found their new election year wedg...,fox news says non whites should only be depict...,republicans found their new election year wedg...,fox news say non white depict separ equal nove...,republican found new elect year wedg issu sinc...,0,Pass
3,Fake_2552,American Intelligence Agents Fear Giving Trum...,Donald Trump is clearly a threat to our nation...,NaN,american intelligence agents fear giving trum...,donald trump is clearly a threat to our nation...,american intelligence agents fear giving trump...,donald trump is clearly a threat to our nation...,american intellig agent fear give trump intel ...,donald trump clearli threat nation secur intel...,0,Pass
4,Fake_14130,MICHELLE OBAMA Wanted Biden To Run So He Could...,Has their ever been a more vengeful and hatefu...,NaN,michelle obama wanted biden to run so he could...,has their ever been a more vengeful and hatefu...,michelle obama wanted biden to run so he could...,has their ever been a more vengeful and hatefu...,michel obama want biden run could beat hillari...,ever veng hate first ladi elit group women occ...,0,Pass


In [12]:
Notebook_end = time.time()
Total_Notebook_Time = round((Notebook_end - Notebook_start)/60.0,)
print(f'This notebook took {Total_Notebook_Time} min to run.')

This notebook took 116 min to run.
